In [ ]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from datasets import Dataset, DatasetDict
import matplotlib.pyplot as plt

# Set random seed for reproducibility
RANDOM_SEED = 42
np.random.seed(RANDOM_SEED)

In [ ]:
# Load the CSV file
df = pd.read_csv('combined_data.csv')
df.head(10)

In [ ]:
# First split: 70% train, 30% temp (for val+test)
train_df, temp_df = train_test_split(
    df, 
    test_size=0.30, 
    random_state=RANDOM_SEED, 
    stratify=df['label']
)

# Second split: 15% val, 15% test (50-50 split of the 30%)
val_df, test_df = train_test_split(
    temp_df, 
    test_size=0.50, 
    random_state=RANDOM_SEED, 
    stratify=temp_df['label']
)

print(f"Train size: {len(train_df)} ({len(train_df)/len(df)*100:.1f}%)")
print(f"Val size: {len(val_df)} ({len(val_df)/len(df)*100:.1f}%)")
print(f"Test size: {len(test_df)} ({len(test_df)/len(df)*100:.1f}%)")

In [ ]:
# Convert to HuggingFace Dataset format
train_dataset = Dataset.from_pandas(train_df[['text', 'label']].reset_index(drop=True))
val_dataset = Dataset.from_pandas(val_df[['text', 'label']].reset_index(drop=True))
test_dataset = Dataset.from_pandas(test_df[['text', 'label']].reset_index(drop=True))

dataset_dict = DatasetDict({
    'train': train_dataset,
    'validation': val_dataset,
    'test': test_dataset
})

dataset_dict

In [ ]:
fig, axes = plt.subplots(1, 3, figsize=(15, 5))

splits = [
    ('Train', train_df),
    ('Validation', val_df),
    ('Test', test_df)
]

colors = ['#ff6b6b', '#4ecdc4']  # Red for non-finance, teal for finance

for idx, (split_name, split_df) in enumerate(splits):
    counts = split_df['label'].value_counts().sort_index()
    percentages = (counts / len(split_df) * 100)
    
    bars = axes[idx].bar(['Non-Domain Related (0)', 'Domain Related (1)'], counts.values, color=colors)
    axes[idx].set_title(f'{split_name} Split (n={len(split_df):,})')
    axes[idx].set_ylabel('Count')
    
    # Add count + percentage labels on bars
    for bar, count, pct in zip(bars, counts.values, percentages.values):
        label = f'{count:,}  ({pct:.1f}%)'
        axes[idx].text(bar.get_x() + bar.get_width()/2, bar.get_height(), 
                       label, ha='center', va='bottom', fontweight='bold', fontsize=10)

plt.suptitle(f'Class Distribution Across Splits - Total: {len(df):,} samples', 
             fontsize=16, fontweight='bold')
plt.tight_layout()
plt.show()

In [ ]:
from transformers import AutoTokenizer, AutoModelForSequenceClassification

MODEL_NAME = "answerdotai/ModernBERT-base"

# Label mappings matching the CSV structure
id2label = {0: "non_finance", 1: "finance"}
label2id = {"non_finance": 0, "finance": 1}

tokenizer = AutoTokenizer.from_pretrained(MODEL_NAME)
model = AutoModelForSequenceClassification.from_pretrained(
    MODEL_NAME, 
    num_labels=2,
    id2label=id2label,
    label2id=label2id
)

model.config

In [ ]:
# Freeze all base model parameters (train only classification head)
for param in model.model.parameters():
    param.requires_grad = False

# Verify: count trainable vs frozen parameters
trainable = sum(p.numel() for p in model.parameters() if p.requires_grad)
frozen = sum(p.numel() for p in model.parameters() if not p.requires_grad)
print(f"Trainable: {trainable:,} | Frozen: {frozen:,}")

In [ ]:
from transformers import DataCollatorWithPadding

def tokenize_fn(batch):
    return tokenizer(batch["text"], truncation=True, max_length=128)

tokenized_dataset = dataset_dict.map(tokenize_fn, batched=True)
data_collator = DataCollatorWithPadding(tokenizer=tokenizer)

tokenized_dataset

In [ ]:
from sklearn.metrics import accuracy_score, precision_recall_fscore_support, roc_auc_score
from scipy.special import softmax

def compute_metrics(eval_pred):
    logits, labels = eval_pred
    preds = np.argmax(logits, axis=-1)
    probs = softmax(logits, axis=-1)
    confidence = np.max(probs, axis=-1)
    
    # Per-class metrics: [non_finance, finance]
    precision, recall, f1, _ = precision_recall_fscore_support(labels, preds, average=None)
    
    # AUC-ROC using probability of finance class
    auc = roc_auc_score(labels, probs[:, 1])
    
    # Confidence stats
    correct_mask = preds == labels
    
    return {
        "accuracy": accuracy_score(labels, preds),
        "auc": auc,
        "precision_non_finance": precision[0],
        "recall_non_finance": recall[0],
        "f1_non_finance": f1[0],
        "precision_finance": precision[1],
        "recall_finance": recall[1],
        "f1_finance": f1[1],
        "confidence_mean": confidence.mean(),
        "confidence_correct": confidence[correct_mask].mean() if correct_mask.any() else 0.0,
        "confidence_wrong": confidence[~correct_mask].mean() if (~correct_mask).any() else 0.0,
    }

In [ ]:
from transformers import TrainingArguments, Trainer, EarlyStoppingCallback

training_args = TrainingArguments(
    output_dir="./layer1_model",
    learning_rate=2e-4,
    num_train_epochs=10,  
    per_device_train_batch_size=32,  
    per_device_eval_batch_size=64,  
    weight_decay=0.05,
    warmup_steps=0.1,
    eval_strategy="epoch",
    save_strategy="epoch",
    load_best_model_at_end=True,
    metric_for_best_model="f1_finance",
    greater_is_better=True,
    logging_steps=50,  
    report_to=[],
)

trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_dataset["train"],
    eval_dataset=tokenized_dataset["validation"],
    data_collator=data_collator,
    compute_metrics=compute_metrics,
    callbacks=[EarlyStoppingCallback(early_stopping_patience=3)],
)

In [ ]:
trainer.train()

In [2]:
# Load the best model from training
from transformers import AutoModelForSequenceClassification, AutoTokenizer
import torch

model = AutoModelForSequenceClassification.from_pretrained("./layer1_model/checkpoint-516")
tokenizer = AutoTokenizer.from_pretrained("./layer1_model/checkpoint-516")

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model = model.to(device)

print(f"✓ Model loaded from ./layer1_model/checkpoint-516")

/Users/isaacanwar/Desktop/Projects/Classification-RAG-gaurdrail/.venv/lib/python3.14/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
Loading weights: 100%|██████████| 138/138 [00:00<00:00, 2243.62it/s, Materializing param=model.layers.21.mlp_norm.weight]    


✓ Model loaded from ./layer1_model/checkpoint-516


In [ ]:
# Evaluate the loaded model on test set
from transformers import Trainer, TrainingArguments

eval_trainer = Trainer(
    model=model,
    args=TrainingArguments(output_dir="./eval_tmp", per_device_eval_batch_size=64),
    data_collator=data_collator,
    compute_metrics=compute_metrics,
)

eval_trainer.evaluate(tokenized_dataset["test"])

In [ ]:
# Threshold analysis
from sklearn.metrics import precision_recall_fscore_support

texts = test_df['text'].tolist()
labels = test_df['label'].values

inputs = tokenizer(texts, truncation=True, max_length=128, padding=True, return_tensors="pt")
inputs = {k: v.to(model.device) for k, v in inputs.items()}

with torch.no_grad():
    probs = torch.softmax(model(**inputs).logits, dim=-1)[:, 1].cpu().numpy()

print(f"{'Threshold':<10} {'Precision':<12} {'Recall':<12} {'F1':<12}")
print("-" * 46)
for t in [0.3, 0.4, 0.5, 0.6, 0.7, 0.8, 0.9]:
    preds = (probs >= t).astype(int)
    p, r, f1, _ = precision_recall_fscore_support(labels, preds, average=None)
    print(f"{t:<10} {p[1]:<12.1%} {r[1]:<12.1%} {f1[1]:<12.1%}")

In [3]:
import torch

query = "can you explain that?"  # <-- break it buddy

inputs = tokenizer(query, return_tensors="pt", truncation=True, max_length=128)
inputs = {k: v.to(model.device) for k, v in inputs.items()}

with torch.no_grad():
    outputs = model(**inputs)
    probs = torch.softmax(outputs.logits, dim=-1)[0]

finance_prob = probs[1].item()
prediction = "✅ You Chillin ✅" if finance_prob >= 0.4 else "❌❌ BLOCKED ❌❌"

print(f"Query: {query}")
print(f"Result: {prediction}")
print(f"Confidence: {max(finance_prob, 1-finance_prob):.1%}")

Query: can you explain that?
Result: ❌❌ BLOCKED ❌❌
Confidence: 97.3%
